# Test: the impact of an AGN power-law continuum
---
What effect do emission lines have upon the recovery of the mass-weighted age of the stellar population? In this notebook, we will look at the effects of 
* emission lines vs. no emission lines;
* broad emission lines vs. narrow emission lines;
* complex, multi-component emission lines vs. simple single-component emission lines
upon the derived mass-weighted age.

In [13]:
%matplotlib widget

In [14]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
display(HTML("<style>.output_result { max-width:75% !important; }</style>"))

In [15]:
import numpy as np
from numpy.random import RandomState
from time import time 
from tqdm.notebook import tqdm
import multiprocessing
import pandas as pd

from astropy.io import fits

from ppxftests.run_ppxf import run_ppxf
from ppxftests.ssputils import load_ssp_templates
from ppxftests.mockspec import load_sfh, create_mock_spectrum, calculate_mw_age
from ppxftests.ppxf_plot import plot_sfh_mass_weighted

import matplotlib.pyplot as plt
plt.ion()
plt.close("all")

from IPython.core.debugger import Tracer

### Defining the AGN continuum
---
In this work, we approximate the AGN continuum as a simple power law,

$$F_{\rm AGN}(\lambda) = F_0 \lambda^{-\alpha_{\lambda}}$$

where the exponent $\alpha_{\lambda} = 2 -\alpha_{\nu}$ where $\alpha_{\nu}$ can range from 1.2 - 2.0 (Groves et al. 2004). 

To determine reasonable values for $F_0$, we use the tight correlation between the total H$\beta$ luminosity, $L_{\rm H\beta}$, and $L_{\rm NT}$, which is defined as the total luminosity of the AGN continuum between 3000 Å and 9500 Å (Neugebauer et al. 1979, Yee 1980), i.e.,

$$L_{\rm NT} = \int^{9500\,Å}_{3000\,Å} F_{\rm AGN}(\lambda) d\lambda 
= \int^{9500\,Å}_{3000\,Å} F_0 \lambda^{-\alpha_{\lambda}} d\lambda 
= \frac{F_0}{1 - \alpha_\lambda}\left(9500^{1 - \alpha_\lambda} - 3000^{1 - \alpha_\lambda} \right)$$

where $F_{\rm AGN}(\lambda)$ is the AGN continuum. In a sample of AGN including quasars, broad & narrow-line radio galaxies and Seyfert 1 & 2 galaxies, Yee (1980) observed a tight correlation such that 

$$L_{\rm NT} \approx 80 L_{\rm H\beta}$$

*Note: equation from [Peterson textbook. eqn. 5.53, p. 90](https://books.google.com.au/books?id=ok4EFlPMStwC&pg=PA90&lpg=PA90&dq=Lnt+agn+continuum&source=bl&ots=QfVvXob4vM&sig=ACfU3U0x69gKrkN-lALkIu0EROAUh1-1vw&hl=en&sa=X&ved=2ahUKEwjW_8fPvqv1AhWXTWwGHarhALcQ6AF6BAgXEAM#v=onepage&q=Lnt%20agn%20continuum&f=false).* 
Referring to table 2 of Dopita et al. (2015), $\log_{10} L_{\rm H\beta} \sim 39 - 42$ in the NLRs of S7 galaxies.
We therefore use our adopted $L_{\rm H\beta}$ in the emission lines to constrain the strength of the AGN continuum in our mock spectra, so that 

$$ F_0 = \frac{80 L_{\rm H\beta}({1 - \alpha_\lambda})}{\left(9500^{1 - \alpha_\lambda} - 3000^{1 - \alpha_\lambda} \right)} $$



In [16]:
###########################################################################
# Defining the AGN continuum using the method of Yee 1980
###########################################################################
L_NT = 1e41

fig, ax = plt.subplots(nrows=1, ncols=1)
for alpha_nu in [0.3, 0.5, 0.7, 1.1, 1.2, 1.3, 1.5, 1.7, 2.0]:
    # Compute the continuum normalisation
    alpha_lambda = 2 - alpha_nu
    F_lambda_0 = L_NT * (1 - alpha_lambda) / (9500**(1 - alpha_lambda) - 3000**(1 - alpha_lambda))

    # Compute the continuum
    lambda_vals_A = np.linspace(3000, 9500, 1e3)
    F_lambda = F_lambda_0 * lambda_vals_A**(-alpha_lambda)

    ax.plot(lambda_vals_A, F_lambda, label=r"$\alpha_\nu =$" + f"{alpha_nu:.1f}")
    ax.set_xlabel("$\lambda$ (Å)")
    ax.set_ylabel("$F(\lambda)$ (erg/s/Å)")
ax.autoscale(axis="x", tight=True, enable=True)
ax.legend()
ax.axvline(3500, color="grey")
ax.axvline(7000, color="grey")
ax.set_title(r"AGN power-law continuum ($L_{\rm NT} = 10^{41}\,\rm erg\,s^{-1}$)")



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'AGN power-law continuum ($L_{\\rm NT} = 10^{41}\\,\\rm erg\\,s^{-1}$)')

In [17]:
###########################################################################
# Settings
###########################################################################
isochrones = "Padova"
SNR = 100
sigma_star_kms = 250
z = 0.01

In [18]:
###########################################################################
# Generate the input SFH
###########################################################################
# Load the stellar templates so we can get the age & metallicity dimensions
_, _, metallicities, ages = load_ssp_templates(isochrones)
N_ages = len(ages)
N_metallicities = len(metallicities)

# Load a realistic SFH
sfh_mw_original = load_sfh(99, plotit=True)

# Compute the mean mass-weighted age below 1 Gyr
log_age_mw_original, log_age_mw_original_idx = calculate_mw_age(sfh_mw_original, age_thresh=1e9, ages=ages)

# Create the spectrum here, just so we can check it looks OK
spec_nolines, spec_nolines_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_original,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=True)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
# Helper function for multiprocessing
def ppxf_helper(args):
    # Unpack arguments
    seed, spec, spec_err, lambda_vals_A = args
    
    # Add "extra" noise to the spectrum
    rng = RandomState(seed)
    noise = rng.normal(scale=spec_err)
    spec_noise = spec + noise

    # This is to mitigate the "edge effects" of the convolution with the LSF
    spec_noise[0] = -9999
    spec_noise[-1] = -9999

    # Run ppxf
    pp = run_ppxf(spec=spec_noise, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                  z=z, ngascomponents=1,
                  regularisation_method="none", 
                  isochrones="Padova",
                  fit_gas=False, tie_balmer=True,
                  plotit=False, savefigs=False, interactive_mode=False)
    return pp

# Helper function so that we can run ppxf on different inputs.
def helper_fn(spec, spec_err, lambda_vals_A,
              niters=100, nthreads=20, ngascomponents=1):  

    ###########################################################################
    # Run ppxf WITHOUT regularisation, using a MC approach
    ###########################################################################
    # Input arguments
    seeds = list(np.random.randint(low=0, high=100 * niters, size=niters))
    args_list = [[s, spec, spec_err, lambda_vals_A] for s in seeds]

    # Run in parallel
    print(f"Running ppxf on {nthreads} threads...")
    t = time()
    with multiprocessing.Pool(nthreads) as pool:
        pp_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
    print(f"Elapsed time in ppxf: {time() - t:.2f} s")

    ###########################################################################
    # Run ppxf WITH regularisation
    ###########################################################################
    t = time()
    pp_regul = run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                        z=z, ngascomponents=ngascomponents,
                        regularisation_method="auto",
                        isochrones="Padova",
                        fit_gas=False, tie_balmer=True,
                        delta_regul_min=1, regul_max=5e4, delta_delta_chi2_min=5,
                        plotit=False, savefigs=False, interactive_mode=False)
    print(f"Total time in run_ppxf: {time() - t:.2f} seconds")

    ###########################################################################
    # Compute the mass-weighted age 
    ###########################################################################
    log_age_mw_regul, log_age_mw_regul_idx = calculate_mw_age(pp_regul.weights_mass_weighted, age_thresh=1e9, ages=ages)

    log_age_mw_list = []
    for pp in pp_list:
        log_age_mw, _ = calculate_mw_age(pp.weights_mass_weighted, age_thresh=1e9, ages=ages)
        log_age_mw_list.append(log_age_mw)

    log_age_mw_MC = np.nanmean(log_age_mw_list)
    log_age_mw_MC_err = np.nanstd(log_age_mw_list)
    # For plotting purposes, figure out the approx. index of these ages in the age array
    log_age_mw_MC_idx = (log_age_mw_MC - np.log10(ages[0])) / (np.log10(ages[1]) - np.log10(ages[0]))
    log_age_mw_MC_err_idx = log_age_mw_MC_err / (np.log10(ages[1]) - np.log10(ages[0]))

    print("------------------------------------------------------------")
    print(f"MC method:    mean mass-weighted age (log yr) = {np.nanmean(log_age_mw_list):.2f} ± {np.nanstd(log_age_mw_list):.2f}")
    print(f"Regul method: mean mass-weighted age (log yr) = {log_age_mw_regul:.2f}")
    print(f"Input value:  mean mass-weighted age (log yr) = {log_age_mw_original:.2f})")
    print("------------------------------------------------------------")

    ###########################################################################
    # COMPARE THE INPUT AND OUTPUT
    ###########################################################################
    sfh_fit_mw_list = []
    sfh_fit_lw_list = []
    sfh_fit_mw_1D_list = []
    sfh_fit_lw_1D_list = []

    for pp in pp_list:
        sfh_fit_mw_list.append(pp.weights_mass_weighted)
        sfh_fit_lw_list.append(pp.weights_light_weighted)
        sfh_fit_mw_1D_list.append(np.nansum(pp.weights_mass_weighted, axis=0))
        sfh_fit_lw_1D_list.append(np.nansum(pp.weights_light_weighted, axis=0))

    # Compute the mean SFH 
    sfh_fit_mw_mean = np.nansum(np.array(sfh_fit_mw_list), axis=0) / len(sfh_fit_mw_list)
    sfh_fit_lw_mean = np.nansum(np.array(sfh_fit_lw_list), axis=0) / len(sfh_fit_lw_list)
    sfh_fit_mw_1D_mean = np.nansum(sfh_fit_mw_mean, axis=0)
    sfh_fit_lw_1D_mean = np.nansum(sfh_fit_lw_mean, axis=0)

    sfh_fit_mw_1D_regul = np.nansum(pp_regul.weights_mass_weighted, axis=0)
    sfh_fit_lw_1D_regul = np.nansum(pp_regul.weights_light_weighted, axis=0)

    # Plot the mass-weighted weights, summed over the metallicity dimension
    for log_scale in [True]:
        # Create new figure 
        fig = plt.figure(figsize=(13, 4))
        ax = fig.add_axes([0.1, 0.2, 0.7, 0.7])
        ax.set_title(f"Mass-weighted template weights (SNR = {SNR:d})")

        # Plot the SFHs from each ppxf run, plus the "truth" SFH
        ax.fill_between(range(N_ages), sfh_mw_1D_original, step="mid", alpha=1.0, color="cornflowerblue", label="Input SFH")
        for jj in range(niters):
            ax.step(range(N_ages), sfh_fit_mw_1D_list[jj], color="pink", alpha=0.2, where="mid", linewidth=0.25, label="ppxf fits (MC simluations)" if jj == 0 else None)
        ax.step(range(N_ages), sfh_fit_mw_1D_mean, color="red", where="mid", label="Mean ppxf fit (MC simulations)")
        ax.step(range(N_ages), sfh_fit_mw_1D_regul, color="lightgreen", where="mid", label="ppxf fit (regularised)")

        # Plot horizontal error bars indicating the mean mass-weighted age from (a) the MC simulations and (b) the regularised fit 
        y = 10**(0.9 * np.log10(ax.get_ylim()[1])) if log_scale else 0.9 * ax.get_ylim()[1]
        ax.errorbar(x=log_age_mw_regul_idx, y=y, xerr=0, yerr=0, 
                    marker="D", mfc="lightgreen",mec="lightgreen",  ecolor="lightgreen",  
                    label="MW age ($< 1$ Gyr) (regularised fit)")
        ax.errorbar(x=log_age_mw_MC_idx, y=y, xerr=log_age_mw_MC_err_idx, yerr=0, 
                    marker="D", mfc="red", mec="red", ecolor="red",
                    label="MW age ($< 1$ Gyr) (MC simulations)")
        ax.errorbar(x=log_age_mw_original_idx, y=y, xerr=0, yerr=0, 
                    marker="D", mfc="cornflowerblue", mec="cornflowerblue", ecolor="cornflowerblue", 
                    label="MW age ($< 1$ Gyr) (input)")

        # Decorations 
        ax.set_xticks(range(N_ages))
        ax.set_xlabel("Age (Myr)")
        ax.set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical", fontsize="x-small")
        ax.autoscale(axis="x", enable=True, tight=True)
        ax.set_ylim([1, None])
        ax.set_ylabel(r"Template weight ($\rm M_\odot$)")
        ax.legend(fontsize="x-small", loc="center left", bbox_to_anchor=(1.01, 0.5))
        ax.set_xlabel("Age (Myr)")
        ax.set_yscale("log") if log_scale else None
        

In [37]:
###########################################################################
# Case 0: No AGN power law continuum, no emission lines
###########################################################################
spec, spec_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_original,
    agn_continuum=False, L_NT=1e42, alpha_nu=1.5,
    isochrones=isochrones, z=z, SNR=1e3, sigma_star_kms=sigma_star_kms,
    plotit=True)

helper_fn(spec, spec_err, lambda_vals_A, niters=100, nthreads=20, ngascomponents=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Running ppxf on 20 threads...



Elapsed time in ppxf: 182.76 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 4.63 s
----------------------------------------------------
Iteration 1: Scaling noise by 9.9891...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 64.23 s
Iteration 1: optimal regul = 0.00; Δm = 5.72257; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 95.079
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 60.94 s
Iteration 2: optimal regul = -1000.00; Δm = 0; Δregul = 100.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 95.079
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 51.02 s
Iteration 3: optimal regul = -1200.00; Δm = 0; Δregul = 20.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
###########################################################################
# Case 1: AGN power law continuum, no emission lines
###########################################################################
spec, spec_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_original,
    agn_continuum=True, L_NT=1e42, alpha_nu=1.5,
    isochrones=isochrones, z=z, SNR=1e3, sigma_star_kms=sigma_star_kms,
    plotit=True)

helper_fn(spec, spec_err, lambda_vals_A, niters=100, nthreads=20, ngascomponents=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Running ppxf on 20 threads...



Elapsed time in ppxf: 317.24 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 4.67 s
----------------------------------------------------
Iteration 1: Scaling noise by 18.8123...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 134.62 s
Iteration 1: optimal regul = 0.00; Δm = 0.00219215; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 95.079
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 91.89 s
Iteration 2: optimal regul = 100.00; Δm = 3.88276e+10; Δregul = 100.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 57.384
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 101.10 s
Iteration 3: optimal regul = 20.00; Δm = 1.21465e+10; Δregul = 20.00 (Δregul_min = 1.0

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:103: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### When an AGN continuum is included, the ppxf fit is still quite good
---
Despite the fact that the recovered SFH isn't accurate when an AGN continuum is included, ppxf is still able to acheive a good fit, with minimal residuals.

In [89]:
plt.close("all")

In [147]:
###############################################################################
# Run ppxf once just to see what the fit looks like
###############################################################################
spec_noAGN, spec_noAGN_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_original,
    agn_continuum=False,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=True)

spec_AGN, spec_AGN_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_original,
    agn_continuum=True, L_NT_erg_s=1e42, alpha_nu=1.5,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [107]:
###############################################################################
# ppxf: without AGN continuum
###############################################################################
pp = run_ppxf(spec=spec_noAGN, spec_err=spec_noAGN_err, lambda_vals_A=lambda_vals_A,
              z=z, ngascomponents=1,
              regularisation_method="none", 
              isochrones="Padova",
              fit_gas=False, tie_balmer=True,
              plotit=True, savefigs=False, interactive_mode=False)
plt.gcf().suptitle("No AGN continuum")

###############################################################################
# ppxf: with AGN continuum
###############################################################################
pp = run_ppxf(spec=spec_AGN, spec_err=spec_AGN_err, lambda_vals_A=lambda_vals_A,
              z=z, ngascomponents=1,
              regularisation_method="none", 
              isochrones="Padova",
              fit_gas=False, tie_balmer=True,
              plotit=True, savefigs=False, interactive_mode=False)
plt.gcf().suptitle("With AGN continuum")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'With AGN continuum')

### What effect does the *strength* of the AGN continuum have on the recovered SFH?
---
Using an MC approach to estimate the mass-weighted age of the stellar population <1 Gyr, we will run ppxf on spectra with the same underlying SFH, but an AGN continuum that varies in strength (parameterised by $L_{\rm NT}$), keeping $\alpha_\nu$ fixed.

In [32]:
alpha_nu = 1.5  # What is a typical value for low-z Seyfert galaxies?
log_L_NT_vals = [0] + list(np.linspace(41, 44, 7))

In [30]:
# Figure for plotting & comparing spectra
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 4))
ax.set_xlabel("Observed wavelength (Å)")
ax.set_ylabel(r"$F_\lambda(\lambda)\,(erg\,s^{-1}\,Å^{-1})$")

for ll, log_L_NT in enumerate(log_L_NT_vals):
    ###########################################################################
    # Create spectrum
    ###########################################################################
    L_NT = 10**log_L_NT
    spec, spec_err, lambda_vals_A = create_mock_spectrum(
        sfh_mass_weighted=sfh_mw_original,
        agn_continuum=True if ll > 0 else False, L_NT_erg_s=L_NT, alpha_nu=alpha_nu,
        isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
        plotit=False)

    # Add to plot 
    ax.errorbar(x=lambda_vals_A, y=spec, yerr=spec_err, 
                label=r"$L_{\rm NT} = 10^{%.1f} \,\rm erg\,s^{-1}$" % log_L_NT if ll > 0 else "No AGN continuum")

ax.legend()
ax.set_yscale("log")
ax.autoscale(axis="x", tight=True, enable=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
###############################################################################
# The effect of the slope & strength of the AGN continuum on the recovered SFH
###############################################################################
# Run ppxf (MC runs only, to save time) on spectra with the same underlying SFH, but an AGN continuum varying in strength.
# Later on, repeat, but vary the power-law index for a "typical" Hbeta luminosity.
# DataFrame for storing results
df = pd.DataFrame()

# ppxf settings
niters = 100
nthreads = 20

for ll, log_L_NT in enumerate(log_L_NT_vals):
    ###########################################################################
    # Create spectrum
    ###########################################################################
    L_NT = 10**log_L_NT
    spec, spec_err, lambda_vals_A = create_mock_spectrum(
        sfh_mass_weighted=sfh_mw_original,
        agn_continuum=True if ll > 0 else False, L_NT_erg_s=L_NT, alpha_nu=alpha_nu,
        isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
        plotit=False)
      
    # Compute mass-weighted mean age of the stellar pop. < 1 Gyr old
    log_age_mw_original, log_age_mw_original_idx = calculate_mw_age(sfh_mw_original, age_thresh=1e9, ages=ages)
    
    ###########################################################################
    # Run ppxf WITHOUT regularisation, using a MC approach
    ###########################################################################
    # Input arguments
    seeds = list(np.random.randint(low=0, high=100 * niters, size=niters))
    args_list = [[s, spec, spec_err, lambda_vals_A] for s in seeds]

    # Run in parallel
    print(f"Running ppxf on {nthreads} threads...")
    t = time()
    with multiprocessing.Pool(nthreads) as pool:
        pp_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
    print(f"Elapsed time in ppxf: {time() - t:.2f} s")

    ###########################################################################
    # Compute the mass-weighted age from the MC runs
    ###########################################################################
    log_age_mw_list = []
    for pp in pp_list:
        log_age_mw, _ = calculate_mw_age(pp.weights_mass_weighted, age_thresh=1e9, ages=ages)
        log_age_mw_list.append(log_age_mw)
    log_age_mw_MC = np.nanmean(log_age_mw_list)
    log_age_mw_MC_err = np.nanstd(log_age_mw_list)

    ###########################################################################
    # Add variables to DataFrame
    ###########################################################################
    df = df.append({
        "AGN continuum?": True if ll > 0 else False,
        "log L_NT": log_L_NT,
        "alpha_nu": alpha_nu,
        "log mass-weighted mean age <1Gyr (input)": log_age_mw_original,
        "log mass-weighted mean age <1Gyr (MC)": log_age_mw_MC,
        "log mass-weighted mean age <1Gyr (MC) error": log_age_mw_MC_err,
    }, ignore_index=True)


Running ppxf on 20 threads...



Elapsed time in ppxf: 180.09 s
Running ppxf on 20 threads...



Elapsed time in ppxf: 188.84 s
Running ppxf on 20 threads...



Elapsed time in ppxf: 197.97 s
Running ppxf on 20 threads...



Elapsed time in ppxf: 193.92 s
Running ppxf on 20 threads...



Elapsed time in ppxf: 196.40 s
Running ppxf on 20 threads...



Elapsed time in ppxf: 239.25 s
Running ppxf on 20 threads...



Elapsed time in ppxf: 286.35 s
Running ppxf on 20 threads...



Elapsed time in ppxf: 459.68 s


In [39]:
########################################################
# Plot the recovered mass-weighted mean age vs. L_NT
########################################################
fig = plt.figure(figsize=(10, 5))
ax = fig.add_axes([0.1, 0.15, 0.5, 0.75])

# Actual value
ax.axhline(df["log mass-weighted mean age <1Gyr (input)"].unique()[0], color="gray", label="Actual value")

# MC run w/o AGN continuum
cond = df["AGN continuum?"] == False
ax.axhspan(ymin=df.loc[cond, "log mass-weighted mean age <1Gyr (MC)"].values[0] - df.loc[cond, "log mass-weighted mean age <1Gyr (MC) error"].values[0],
           ymax=df.loc[cond, "log mass-weighted mean age <1Gyr (MC)"].values[0] + df.loc[cond, "log mass-weighted mean age <1Gyr (MC) error"].values[0],
           color="pink", alpha=0.5)
ax.axhline(df.loc[cond, "log mass-weighted mean age <1Gyr (MC)"].values[0], color="pink", label="MC estimate (no AGN continuum)")

# Results from MC runs
cond = df["AGN continuum?"] == True
ax.errorbar(x=df.loc[cond, "log L_NT"].values, 
            y=df.loc[cond, "log mass-weighted mean age <1Gyr (MC)"].values, 
            yerr=df.loc[cond, "log mass-weighted mean age <1Gyr (MC) error"].values,
            linestyle="none", marker="D", label="MC estimate", zorder=999)

# Decorations
ax.set_xlabel(r"$\log_{10} \,L_{\rm NT}$")
ax.set_ylabel(r"Mass-weighted mean age < 1 Gyr (log yr)")
ax.grid()
ax.legend(bbox_to_anchor=[1.05, 0.5], loc="center left")
ax.set_title(r"Effect of AGN continuum ($\alpha_\nu = %.1f$)" % alpha_nu)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Effect of AGN continuum ($\\alpha_\\nu = 1.5$)')

In [117]:
plt.close("all")

### What effect does the *slope* of the AGN continuum have on the recovered SFH?
---
Using an MC approach to estimate the mass-weighted age of the stellar population <1 Gyr, we will run ppxf on spectra with the same underlying SFH, but an AGN continuum that varies in slope (parameterised by $\alpha_\nu$), keeping $L_{\rm NT}$ fixed.

In [51]:
alpha_nu_vals = [0] + list(np.linspace(0.3, 2.1, 5))  # What is a typical value for low-z Seyfert galaxies?
log_L_NT = 43

In [52]:
alpha_nu_vals

[0, 0.3, 0.75, 1.2, 1.6500000000000001, 2.1]

In [54]:
###############################################################################
# The effect of the exponent of the AGN continuum on the recovered SFH
###############################################################################
# Run ppxf (MC runs only, to save time) on spectra with the same underlying SFH, but an AGN continuum varying in power-law index.


# DataFrame for storing results
df = pd.DataFrame()

# Figure for plotting & comparing spectra
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 4))
ax.set_xlabel("Observed wavelength (Å)")
ax.set_ylabel(r"$F_\lambda(\lambda)\,(erg\,s^{-1}\,Å^{-1})$")

# ppxf settings
niters = 100
nthreads = 20

for aa, alpha_nu in enumerate(alpha_nu_vals):
        ###########################################################################
        # Create spectrum
        ###########################################################################
        spec, spec_err, lambda_vals_A = create_mock_spectrum(
            sfh_mass_weighted=sfh_mw_original,
            agn_continuum=True if aa > 0 else False, L_NT_erg_s=10**log_L_NT, alpha_nu=alpha_nu,
            isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
            plotit=False)

        # Compute mass-weighted mean age of the stellar pop. < 1 Gyr old
        log_age_mw_original, log_age_mw_original_idx = calculate_mw_age(sfh_mw_original, age_thresh=1e9, ages=ages)

        # Add to plot 
        ax.errorbar(x=lambda_vals_A, y=spec, yerr=spec_err, 
                    label=r"$L_{\rm NT} = 10^{%.1f} \,\rm erg\,s^{-1}, \alpha_\nu = %.1f$" % (log_L_NT, alpha_nu) if aa > 0 else "No AGN continuum")

        ###########################################################################
        # Run ppxf WITHOUT regularisation, using a MC approach
        ###########################################################################
        # Input arguments
        seeds = list(np.random.randint(low=0, high=100 * niters, size=niters))
        args_list = [[s, spec, spec_err, lambda_vals_A] for s in seeds]

        # Run in parallel
        print(f"Running ppxf on {nthreads} threads...")
        t = time()
        with multiprocessing.Pool(nthreads) as pool:
            pp_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
        print(f"Elapsed time in ppxf: {time() - t:.2f} s")

        ###########################################################################
        # Compute the mass-weighted age from the MC runs
        ###########################################################################
        log_age_mw_list = []
        for pp in pp_list:
            log_age_mw, _ = calculate_mw_age(pp.weights_mass_weighted, age_thresh=1e9, ages=ages)
            log_age_mw_list.append(log_age_mw)
        log_age_mw_MC = np.nanmean(log_age_mw_list)
        log_age_mw_MC_err = np.nanstd(log_age_mw_list)

        ###########################################################################
        # Add variables to DataFrame
        ###########################################################################
        df = df.append({
            "AGN continuum?": True if aa > 0 else False,
            "log L_NT": log_L_NT,
            "alpha_nu": alpha_nu,
            "log mass-weighted mean age <1Gyr (input)": log_age_mw_original,
            "log mass-weighted mean age <1Gyr (MC)": log_age_mw_MC,
            "log mass-weighted mean age <1Gyr (MC) error": log_age_mw_MC_err,
        }, ignore_index=True)

ax.legend()
ax.set_yscale("log")
ax.autoscale(axis="x", tight=True, enable=True)

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # This is added back by InteractiveShellApp.init_path()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Running ppxf on 20 threads...



Elapsed time in ppxf: 195.83 s
Running ppxf on 20 threads...



Elapsed time in ppxf: 208.30 s
Running ppxf on 20 threads...



Elapsed time in ppxf: 206.95 s
Running ppxf on 20 threads...



Elapsed time in ppxf: 198.27 s
Running ppxf on 20 threads...



Elapsed time in ppxf: 201.14 s
Running ppxf on 20 threads...



Elapsed time in ppxf: 227.49 s


In [55]:
########################################################
# Plot the recovered mass-weighted mean age vs. alpha_nu
########################################################
fig = plt.figure(figsize=(10, 5))
ax = fig.add_axes([0.1, 0.15, 0.5, 0.75])

# Actual value
ax.axhline(df["log mass-weighted mean age <1Gyr (input)"].unique()[0], color="gray", label="Actual value")

# MC run w/o AGN continuum
cond = df["AGN continuum?"] == False
ax.axhspan(ymin=df.loc[cond, "log mass-weighted mean age <1Gyr (MC)"].values[0] - df.loc[cond, "log mass-weighted mean age <1Gyr (MC) error"].values[0],
           ymax=df.loc[cond, "log mass-weighted mean age <1Gyr (MC)"].values[0] + df.loc[cond, "log mass-weighted mean age <1Gyr (MC) error"].values[0],
           color="pink", alpha=0.5)
ax.axhline(df.loc[cond, "log mass-weighted mean age <1Gyr (MC)"].values[0], color="pink", label="MC estimate (no AGN continuum)")

# Results from MC runs
cond = df["AGN continuum?"] == True
ax.errorbar(x=df.loc[cond, "alpha_nu"].values, 
            y=df.loc[cond, "log mass-weighted mean age <1Gyr (MC)"].values, 
            yerr=df.loc[cond, "log mass-weighted mean age <1Gyr (MC) error"].values,
            linestyle="none", marker="D", label="MC estimate", zorder=999)

# Decorations
ax.set_xlabel(r"$\alpha_\nu$")
ax.set_ylabel(r"Mass-weighted mean age < 1 Gyr (log yr)")
ax.grid()
ax.legend(bbox_to_anchor=[1.05, 0.5], loc="center left")
ax.set_title(r"Effect of AGN continuum ($\log_{10} \, L_{\rm NT} = %.1f$)" % log_L_NT)


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Effect of AGN continuum ($\\log_{10} \\, L_{\\rm NT} = 43.0$)')